In [1]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
import numba
import numpy as np
from util import perf_comp_data
from native_util import load
%load_ext cythonmagic
%load_ext version_information
%version_information numpy, Cython, numba, hope

The Cython magic has been move to the Cython package, hence 
`%load_ext cythonmagic` is deprecated; Please use `%load_ext Cython` instead.
You need Cython version >=0.21 to use the Cython magic


/Users/jakeret/Library/Python/2.7/lib/python/site-packages/IPython/core/formatters.py:832: FormatterWarning: JSON expects JSONable list/dict containers, not JSON strings
  FormatterWarning)


Software versions
Python 2.7.10 64bit [GCC 4.2.1 Compatible Apple LLVM 5.1 (clang-503.0.40)]
IPython 3.0.0
OS Darwin 13.4.0 x86_64 i386 64bit
numpy 1.8.1
Cython 0.20.2
numba 0.20.0
hope 0.4.0
Thu Jul 16 17:38:42 2015 CEST

In [2]:
# Pure python version

def pairwise_python(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [3]:
# Numpy python version

def pairwise_numpy(X, D):
    M = X.shape[0]
    for i in range(M):
        D[i, :] = np.sqrt(np.sum((X[i, :] - X[:]) ** 2, axis=1))

In [4]:
# numba version
@numba.jit(nopython=True)
def pairwise_numba(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)

In [5]:
%%cython

cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X, double[:, ::1] D):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)

ERROR: Cell magic `%%cython` not found.


In [6]:
# hope version
@hope.jit
def pairwise_hope(X, D, M, N):
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)


In [7]:
from native_util import load

native_pairwise_mod = load("pairwise")
pairwise_native = native_pairwise_mod.run

running build_ext
building 'pairwise' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -fno-common -dynamic -pipe -Os -fwrapv -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

compile options: '-I/Users/jakeret/workspace/virtualenvs/hope_numba_benchmarks/lib/python2.7/site-packages/numpy/core/include -I/opt/local/Library/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11'
clang: ././src/pairwise.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -L/opt/local/lib -Wl,-headerpad_max_install_names -L/opt/local/lib/db48 ./src/pairwise.o -o ./pairwise.so



In [8]:
X = np.random.random((1000, 3))
D = np.empty((1000, 1000))

In [10]:
D1 = np.empty((1000, 1000))
D2 = np.empty((1000, 1000))
D3 = np.empty((1000, 1000))
D4 = np.empty((1000, 1000))
D5 = np.empty((1000, 1000))
D6 = np.empty((1000, 1000))

pairwise_python(X, D1)
pairwise_numpy(X, D2)
pairwise_numba(X, D3)
#pairwise_cython(X, D4)
pairwise_hope(X, D5, X.shape[0], X.shape[1])
pairwise_native(X, D6, X.shape[0], X.shape[1])

assert np.allclose(D1, D2)
assert np.allclose(D1, D3)
assert np.allclose(D1, D4)
assert np.allclose(D1, D5)
assert np.allclose(D1, D6)


pairwise_hope(float64^2 X, float64^2 D, int64 M, int64 N)
	for i.l in (0.J:M.J) {
		for j.l in (0.J:M.J) {
			new d.D
			d.D = 0.0.D
			for k.l in (0.J:N.J) {
				new tmp.d
				tmp.d = (X.d[i.l, k.l] - X.d[j.l, k.l])
				d.D += (tmp.d * tmp.d)
			}
			D.d[i.l, j.l] = numpy.sqrt(d.D)
		}
	}

Compiling following functions:
pairwise_hope(float64^2 X, float64^2 D, int64 M, int64 N)
running build_ext
building 'pairwise_hope_f82cb8a6ea3aecbde9f728fa85088b51f5bdf7d8ef8b740b9c0b788a_0' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -fno-common -dynamic -pipe -Os -fwrapv -DNDEBUG -g -fwrapv -O3 -Wall

compile options: '-I/Users/jakeret/workspace/virtualenvs/hope_numba_benchmarks/lib/python2.7/site-packages/numpy/core/include -I/opt/local/Library/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11'
clang: /var/folders/k_/ttz9cd4d3kj2kbh3lvtn6tkm0000gn/T/hopen1VeFP/pairwise_hope_f82cb8a6ea3aec

In [11]:
print "naive python"
%timeit pairwise_python(X, D)
print "numpy"
%timeit pairwise_numpy(X, D)
print "numba"
%timeit pairwise_numba(X, D)
print "cython"
#%timeit pairwise_cython(X, D)
print "hope"
%timeit pairwise_hope(X, D, X.shape[0], X.shape[1])
print "native"
%timeit pairwise_native(X, D, X.shape[0], X.shape[1])

naive python
1 loops, best of 3: 5.98 s per loop
numpy
10 loops, best of 3: 38.4 ms per loop
numba
100 loops, best of 3: 4.45 ms per loop
cython
hope
100 loops, best of 3: 7.96 ms per loop
native
100 loops, best of 3: 4.3 ms per loop


In [12]:
from util import perf_comp_data

In [14]:
M, N = X.shape
data_list = 3*["X, D"]+ 2*["X, D, M, N"]
#print data_list
perf_comp_data(["pairwise_python", 
                "pairwise_numpy", 
                "pairwise_numba", 
                #"pairwise_cython", 
                "pairwise_hope",
                "pairwise_native"],
               data_list, rep=10)

function: pairwise_numba      , av. time sec:   0.00467312, min. time sec:   0.00453520, relative:       1.0
function: pairwise_native     , av. time sec:   0.00484359, min. time sec:   0.00457597, relative:       1.0
function: pairwise_hope       , av. time sec:   0.00881946, min. time sec:   0.00793386, relative:       1.9
function: pairwise_numpy      , av. time sec:   0.04122066, min. time sec:   0.03978014, relative:       8.8
function: pairwise_python     , av. time sec:   6.10364544, min. time sec:   5.95258522, relative:    1306.1
